# Winston: A Cat Breed Classifier CNN

**Import Libraries**

In [ ]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from glob import glob
from tqdm import tqdm, trange
import pickle
import os

from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

from keras.models import Model

from keras.utils.vis_utils import plot_model

from PIL import ImageFile
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt

# Helper Functions

In [ ]:
ResNet50_model = ResNet50(weights='imagenet')

# Images to tensors
def get_image_tensor(img_path):
  img_path = img_path.replace('\\', '/')
  img = image.load_img(img_path, target_size=(224, 224))
  img_array = image.img_to_array(img)
  return np.expand_dims(img_array, axis=0)

def images_to_tensor(paths):
  tensor_list = [get_image_tensor(img_path) for img_path in tqdm(paths)]
  return np.vstack(tensor_list)


# For Detecting if image is a cat, UNUSED
def predict_labels(img_path):
  img = preprocess_input(get_image_tensor(img_path))
  return np.argmax(ResNet50_model.predict(img))
# 281 293
def detect_cat(img_path):
  prediction = predict_labels(img_path)
  return ((prediction <= 293) & (prediction >= 281))

# Load Images
Skip if image tensors have been pickled

In [ ]:
def load(path):
  data = load_files(path, shuffle=True)
  cat_img_files = np.array(data['filenames'])
  cat_targets = np_utils.to_categorical(np.array(data['target']), num_classes=12)
  return cat_img_files, cat_targets

print("Loading Test Set")
test_files, test_targets = load('dataset/Testing_Data')
# pickle.dump(test_files, open('dataset/test_files.p', 'wb'))
# pickle.dump(test_targets, open('dataset/test_targets.p', 'wb'))

print("Loading Validation Set")
validation_files, validation_targets = load('dataset/Validation_Data')
# pickle.dump(validation_files, open('dataset/validation_files.p', 'wb'))
# pickle.dump(validation_targets, open('dataset/validation_targets.p', 'wb'))

print("Loading Train Set")
train_files, train_targets = load('dataset/Training_Data')
# pickle.dump(train_files, open('dataset/train_files.p', 'wb'))
# pickle.dump(train_targets, open('dataset/train_targets.p', 'wb'))

**Dataset Stats**

Skip if image tensors have been pickled

In [ ]:
breeds_list = [os.path.split(path)[-1:][0] for path in sorted(glob("dataset/Training_Data/*"))]
print(f"# breeds: {len(breeds_list)}")
print(f"# total cat images: {len(np.hstack([train_files, validation_files, test_files]))}")
print(f"# training imaqes: {len(train_files)}")
print(f"# test images: {len(test_files)}")
print(f"# validation images: {len(validation_files)}")

# list of breed labels
print("Dumping breed labels")
print(breeds_list)
pickle.dump(breeds_list, open('dataset/classes_list.p', 'wb'))

# Images to Tensors
Skip if image tensors have been pickled

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
train_tensors = images_to_tensor(train_files).astype('float32')/255
validation_tensors = images_to_tensor(validation_files).astype('float32')/255
test_tensors = images_to_tensor(test_files).astype('float32')/255

# Pickle Tensors (Optional)
Skip if image tensors have been pickled

In [ ]:
pickle.dump(train_tensors, open('dataset/train_tensors.p', 'wb'), protocol=4)
pickle.dump(validation_tensors, open('dataset/validation_tensors.p', 'wb'), protocol=4)
pickle.dump(test_tensors, open('dataset/test_tensors.p', 'wb'), protocol=4)

**Run if tensors were pickled**

In [ ]:
train_tensors = pickle.load(open('dataset/train_tensors.p', 'rb'))
validation_tensors = pickle.load(open('dataset/validation_tensors.p', 'rb'))
test_tensors = pickle.load(open('dataset/test_tensors.p', 'rb'))

In [ ]:
print(f"# Train: {len(train_tensors)}")
print(f"# Validation: {len(validation_tensors)}")
print(f"# Test: {len(test_tensors)}")

# Build Model

In [ ]:
model = Sequential()

# Building the model

model.add(Conv2D(input_shape=train_tensors.shape[1:], filters=6, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=96, kernel_size=5, strides= 2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))
model.add(Conv2D(filters=324, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=756, kernel_size=5, strides= 2, activation='relu'))
model.add(Dropout(0.3))
model.add(GlobalAveragePooling2D())
#model.add(Flatten())
model.add(Dense(12, activation='softmax'))

model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 200

### **Train**

In [ ]:
!mkdir saved_models
checkpointer = ModelCheckpoint(filepath=f"dataset/saved_models/weights.12breeds._f_{epochs}.best.hdf5", verbose=1, save_best_only=True)

history = model.fit(
          train_tensors, train_targets,
          validation_data=(validation_tensors, validation_targets),
          epochs=epochs, batch_size=64, callbacks=[checkpointer], verbose=1
          )

[ OR ]

**Load pre-trainded model**

In [ ]:
model.load_weights(f'dataset/saved_models/weights.12breeds._f_{epochs}.best.hdf5')

# Visualize

### Activation Layers

In [ ]:
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(np.expand_dims(test_tensors[50], axis=0))
 
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*5,col_size*3))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='gray')
            activation_index += 1

display_activation(activations, 8, 8, 3)

### Image Tensor

In [ ]:
plt.imshow(test_tensors[50][:,:,0])

### Layer Chart

In [ ]:
# Pydot library required
# !pip install pydot
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

### Stats
* Must be trained to generate

In [ ]:
print(history.history.keys())
# Mean Absolute Error
plt.plot(history.history['loss'], label='MAE (testing data)')
plt.plot(history.history['val_loss'], label='MAE (validation data)')
plt.title('Mean Absolute Error')
plt.ylabel('MAE value')
plt.xlabel('No. epoch')
plt.legend(loc="upper right")
plt.show()
plt.clf()

# Accuracy
plt.plot(history.history['accuracy'], label='Acc (testing data)')
plt.plot(history.history['val_accuracy'], label='Acc (validation data)')
plt.title('Accuracy')
plt.ylabel('Acc value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()
plt.clf()

# TEST

In [ ]:
cat_preds = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]
test_accuracy = 100*np.sum(np.array(cat_preds)==np.argmax(test_targets, axis=1))/len(cat_preds)
print('Test accuracy: %.4f%%' % test_accuracy)

## Predictor

In [ ]:
img_path = input("Enter image path: ")    # base dir is that of the notebook

try:
    img_tensor = get_image_tensor(img_path)
    
    breed_labels = pickle.load(open('dataset/classes_list.p', 'rb'))
    
    prediction = model.predict_classes(img_tensor, verbose=1)
    print(f"It's a(n) {breed_labels[prediction[0]]}")
except Exception as e:
    print(f"Failed to load image:\n{e})